## Create a generative AI runbook to resolve security findings

## Module 4 - Using Amazon Bedrock Knowledge Bases

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will complete the following:
- Get a finding from Security Hub
- Use Claude to summarize the finding result
- Use Amazon Bedrock Knowledge Bases to get relevant security policy information
- Use Claude to generate Python code used to remediate the finding with the security policy requirements

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

**In this example, you will get a failed finding from Security Hub for `Ensure IAM password policy requires minimum password length of 14 or greater [IAM.15]`**

<div class="alert alert-block alert-warning">
<b>Action required:</b> Before continuing with this notebook, you must create a <b>Knowledge Base</b> from the console, and upload the password policy document.
</div>

***

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 1: Create an Amazon Bedrock Knowledge Base</h3>
    <p>Follow the steps in the workshop to create an Amazon Bedrock Knowledge Base.</p>
</div>

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 2: Upload password policy and sync data store.</h3>
    <p>Follow the steps in the workshop to upload a password policy markdown file and sync data store.</p>
</div>

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 3: Get a finding from Security Hub</h3>
    <p>In this step, you will get a failed finding from Security Hub. In this example, we use the AWS SDK for Python (boto3) to access the Security Hub API.</p>
    <p>The Python function will return a JSON representation of the finding. You can also view this finding by visiting Security Hub in the AWS console.</p>
</div>

In [ ]:
# Python code from Module 1. Update to search for `Ensure IAM password policy
# requires minimum password length of 14 or greater [IAM.15]`

import boto3

sh = boto3.client('securityhub')


def get_finding(finding_id):
    response = sh.get_findings(
        Filters={
                'ComplianceSecurityControlId': [
                    {
                        'Value': finding_id,
                        'Comparison': 'EQUALS'
                    }
                ],
                'ComplianceStatus':[
                    {
                        'Value': 'FAILED',
                        'Comparison': 'EQUALS'
                    }
                ]
            },
        MaxResults=1

    )
    return response['Findings']


######
# Update get_finding("XXX") with IAM.15
######

sh_finding = get_finding("XXX")

######

print(sh_finding)

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 4: Summarize the finding using Claude Sonnet and an Amazon Bedrock Knowledge Base</h3>
    <p>In this step, you will use the boto3 retrieve and generate API, to prompt Claude to use the Knowledge Base.</p>
</div>

<div class="alert alert-block alert-warning">
<b>Action required:</b> Before continuing with this notebook, you must get the Knowledge Base ID from the console, and upload the password policy document.
</div>

In [ ]:
# Update the knowledge_base_id with the Knowledge Base ID from the
# Amazon Bedrock console.

######
# Update with the Knowledge Base ID from the console.
######

knowledge_base_id = "D0TEXAMPLE"

######

modelId = "global.anthropic.claude-sonnet-4-20250514-v1:0"
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')


def retrieve_and_generate(prompt):

    retrieve_and_generate_api_params = {
        "input": {
            "text": prompt
        },
        "retrieveAndGenerateConfiguration": {
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": knowledge_base_id,
                "modelArn": modelId,
                "generationConfiguration": {
                    "inferenceConfig": {
                        "textInferenceConfig": {
                            "temperature": 0
                        }
                    }
                },
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            }
        }
    }

    response = bedrock_agent_runtime_client.retrieve_and_generate(**retrieve_and_generate_api_params)
    return response


print("Retrieve and generate function defined.")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <p>After loading the function, you can create your first prompt.</p>
</div>

In [ ]:
# Ask Claude Sonnet to summarize the finding and refer to the security
# policy in the knowledge base to provide a recommendation.

prompt = f"""
Summarize the finding, and cite appropriate security policy to remediate
the finding to meet the security policy.

<finding>
{sh_finding}
</finding>

"""

results = retrieve_and_generate(prompt)

print(f"Response: {results['output']['text']}\n")
print(f"Citation: {results['citations'][0]['retrievedReferences'][0]['content']['text']}\n")
print(f"Location: {results['citations'][0]['retrievedReferences'][0]['location']['s3Location']['uri']}\n")

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 5: Use results to get remediation actions</h3>
    <p>In this step, you will use the boto3 converse, to prompt Claude to generate sample code to remediate.</p>
</div>

In [ ]:
bedrock_client = boto3.client(service_name='bedrock-runtime')
inference_config = {"temperature": 0}


def get_completion(prompt):

    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }

    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text']


print("Get completion function defined.")

In [ ]:
# In this prompt we state not to remediate anything else. In testing, Claude
# would consider the rest of the citation from the previous step and remediate
# several password polices requirements.

prompt = f"""
    Review the finding and security policy. Create a minimal Python function
    called remediate_password_policy to remediate the finding to align
    with the organization security policy. Include example usage.
    Do not remediate anything else.

    <finding>
    {results['output']['text']}
    </finding>

"""

response = get_completion(prompt)

print(response)

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 6: Remediate the finding</h3>
    <p>In this step, you will use the sample code created by Claude Sonnet to remediate the finding.</p>
    <p>Carefully review the function. Once you are familiar with how it works, run the function to update the IAM password policy to require a minimum of 64 characters.</p>
</div>

In [ ]:
# Replace the text in this cell with the function you created in step 5.
# Your function may be slightly different. Generative AI responses are non deterministic.

import boto3

def remediate_password_policy():
    """
    Remediate the IAM password policy finding by setting the minimum password
    length to 64 characters as required by the security policy.
    """
    try:
        # Create IAM client
        iam_client = boto3.client('iam')

        # Update the account password policy to require minimum length of 64 characters
        response = iam_client.update_account_password_policy(
            MinimumPasswordLength=64,  # Set minimum password length to 64 characters
            # Keeping other password policy settings unchanged
            # If you need to modify other settings, they would be added here
        )

        print("Successfully updated IAM password policy to require minimum length of 64 characters")
        return True

    except Exception as e:
        print(f"Error updating IAM password policy: {str(e)}")
        return False


# Example usage
if __name__ == "__main__":
    remediate_password_policy()

<div style='background-color:#f0f0f0; padding:10px; border-radius:5px;'>
    <h3>Step 7: Verify the finding</h3>
    <p>In this step, you will verify the password policy has been updated.</p>
</div>

In [ ]:
def check_iam_password_policy_length():
    """
    Retrieves the IAM password policy and returns the minimum password length.

    Returns:
        int: The minimum password length required by the policy
        None: If no password policy is set or an error occurs
    """
    try:
        # Create an IAM client
        iam_client = boto3.client('iam')

        # Get the account password policy
        response = iam_client.get_account_password_policy()

        # Extract and return the minimum password length
        min_length = response['PasswordPolicy'].get('MinimumPasswordLength')
        return min_length

    except iam_client.exceptions.NoSuchEntityException:
        print("No password policy is set for this account.")
        return None
    except Exception as e:
        print(f"Error retrieving password policy: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    min_length = check_iam_password_policy_length()
    if min_length is not None:
        print(f"The minimum password length required is: {min_length} characters")
    else:
        print("Could not determine the minimum password length requirement")